In [1]:
import pandas as pd
import numpy as np 
from pandas.api.types import CategoricalDtype
from datetime import datetime 
import pandas as pd
import numpy as np 
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from googletrans import Translator
from sklearn.preprocessing import normalize

## movies metadata file cleaning

In [2]:
movies_df = pd.read_csv("Data/movies_metadata.csv")

/var/folders/j1/rhs6cm1s6r33j2qkcxx0f8kr0000gn/T/ipykernel_4951/2760051603.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv("Data/movies_metadata.csv")


In [3]:
movies_df.shape

(45466, 24)

In [4]:
movies_df.index.min()
movies_df.index.max()

45465

In [5]:
data_df = movies_df.drop(columns = ["adult","belongs_to_collection","homepage" ,"tagline","status","production_companies","production_countries",
                         "revenue","video"] , inplace= False)

In [6]:
data_df.isnull().sum()

budget                 0
genres                 0
id                     0
imdb_id               17
original_language     11
original_title         0
overview             954
popularity             5
poster_path          386
release_date          87
runtime              263
spoken_languages       6
title                  6
vote_average           6
vote_count             6
dtype: int64

In [7]:
data_df[data_df['vote_average'].isnull()] = 0
data_df[data_df['vote_count'].isnull()] = 0

In [8]:
data_df.isnull().sum()

budget                 0
genres                 0
id                     0
imdb_id               17
original_language     11
original_title         0
overview             954
popularity             0
poster_path          383
release_date          84
runtime              257
spoken_languages       0
title                  0
vote_average           0
vote_count             0
dtype: int64

In [9]:
data_df.drop(columns="budget" , inplace=True)

In [10]:
data_df.isnull().sum()

genres                 0
id                     0
imdb_id               17
original_language     11
original_title         0
overview             954
popularity             0
poster_path          383
release_date          84
runtime              257
spoken_languages       0
title                  0
vote_average           0
vote_count             0
dtype: int64

In [11]:
data_df['overview']= data_df['overview'].fillna(data_df['title'])

In [12]:
data_df[data_df['runtime'].isnull()] = data_df['runtime'].mean()

In [13]:
data_df.drop(columns="poster_path" , inplace=True)

In [14]:
data_df.isnull().sum()

genres                0
id                    0
imdb_id              15
original_language    11
original_title        0
overview              0
popularity            0
release_date         73
runtime               0
spoken_languages      0
title                 0
vote_average          0
vote_count            0
dtype: int64

In [15]:
data_df["original_language"] = data_df['original_language'].fillna(data_df.spoken_languages[1])


In [16]:
data_df.head()

,genres,id,imdb_id,original_language,original_title,overview,popularity,release_date,runtime,spoken_languages,title,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,6.9,2413.0
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,6.5,92.0
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,6.1,34.0
4,"[{'id': 35, 'name': 'Comedy'}]",11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5.7,173.0


In [17]:
data_df.isnull().sum()

genres                0
id                    0
imdb_id              15
original_language     0
original_title        0
overview              0
popularity            0
release_date         73
runtime               0
spoken_languages      0
title                 0
vote_average          0
vote_count            0
dtype: int64

In [18]:
data_df = data_df[pd.to_datetime(data_df["release_date"],errors='coerce').notnull()]
data_df['release_date'] = pd.to_datetime(data_df['release_date'])


In [19]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45130 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   genres             45130 non-null  object        
 1   id                 45130 non-null  object        
 2   imdb_id            45118 non-null  object        
 3   original_language  45130 non-null  object        
 4   original_title     45130 non-null  object        
 5   overview           45130 non-null  object        
 6   popularity         45130 non-null  object        
 7   release_date       45130 non-null  datetime64[ns]
 8   runtime            45130 non-null  float64       
 9   spoken_languages   45130 non-null  object        
 10  title              45130 non-null  object        
 11  vote_average       45130 non-null  float64       
 12  vote_count         45130 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(9)
memory usage: 4.8+ MB


In [20]:
data_df[data_df['release_date'].isnull()] = data_df['release_date'].mean()

In [21]:
data_df.isnull().sum()

genres                0
id                    0
imdb_id              12
original_language     0
original_title        0
overview              0
popularity            0
release_date          0
runtime               0
spoken_languages      0
title                 0
vote_average          0
vote_count            0
dtype: int64

In [22]:
data_df[data_df['imdb_id'].isnull()] = "Unknown"

In [23]:
#sanity check
data_df.isnull().sum()

genres               0
id                   0
imdb_id              0
original_language    0
original_title       0
overview             0
popularity           0
release_date         0
runtime              0
spoken_languages     0
title                0
vote_average         0
vote_count           0
dtype: int64

In [24]:
data_df.head(1)

,genres,id,imdb_id,original_language,original_title,overview,popularity,release_date,runtime,spoken_languages,title,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30 00:00:00,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0


## ratings file cleaning

In [25]:
ratings_df = pd.read_csv("Data/ratings.csv")

In [26]:
ratings_df.head(2)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435


In [27]:
#we're gonna drop the timestamp col
ratings_df.drop(columns = "timestamp", inplace=True)

In [28]:
ratings_df.isnull().sum()

userId     0
movieId    0
rating     0
dtype: int64

Changing name of the `id` col to `movieID` to join the review table with movies.

In [29]:
movies =data_df.rename(columns = {'id' : 'movieId'})

In [30]:
#add the avg rating for each movie to the Dataframe
ratings = ratings_df.groupby("movieId")["rating"].mean().round(2)

In [31]:
ratings

movieId
1         3.89
2         3.24
3         3.18
4         2.88
5         3.08
          ... 
176267    4.00
176269    3.50
176271    5.00
176273    1.00
176275    3.00
Name: rating, Length: 45115, dtype: float64

In [32]:
lang_list=[]
for lang in movies['spoken_languages']:
    lang = lang.strip("[").strip("]")
    lang_list.append(lang.split(','))


In [33]:
languages = [] #list of lists


for i in range(len(lang_list)):
    
    ls=[] #list of dicts
    
    for item in lang_list[i]:
        temp = {}
        item = item.replace("'", "").replace("{", "").replace("}", "")
        if len(item.split(":"))==2:
            key, value = item.split(":")
            key = key.strip()
            value = value.strip()
            temp[key] = value
   
            ls.append(temp)
    languages.append(ls)      

In [34]:
len(languages)

45130

In [35]:
movies.shape

(45130, 13)

In [36]:
langs =[]
for i in range(len(languages)):
    temp =[]
    
    for idx,dct in enumerate(languages[i]):
        
        for key,val in dct.items():
            if key =='name':
                temp.append(val)
                
   
    #list of temps
    langs.append(temp)
  
movies['language'] = langs
movies['lang_list'] = langs

In [37]:
#converting language column from list ot strings
movies['language'] = [','.join(map(str, l)) for l in movies['language']]


In [38]:
#dropping the spoken languages col
movies.drop(columns = 'spoken_languages',inplace=True)

In [39]:
genre_list=[]
for genre in movies['genres']:
    genre = genre.strip("[").strip("]")
    genre_list.append(genre.split(','))


In [40]:
result = [] #list of lists


for i in range(len(genre_list)):
    
    ls=[] #list of dicts
    
    for item in genre_list[i]:
        temp = {}
        item = item.replace("'", "").replace("{", "").replace("}", "")
        if len(item.split(":"))==2:
            key, value = item.split(":")
            key = key.strip()
            value = value.strip()
            temp[key] = value
   
            ls.append(temp)
    result.append(ls)        
        

In [41]:
len(genre_list)

45130

In [42]:
len(result)

45130

In [43]:
final_movies_df = movies.copy()


In [44]:
genres_names = []
for item in result:
    for dct in item:
        if 'name' not in dct.keys():
            item.remove(dct)
        

In [45]:
gen_list = []
for item in result:
    genres = []
    for dct in item:
        k,val = list(dct.items())[0]
        genres.append(val)
        
    gen_list.append(genres)
    


In [46]:
len(gen_list)

45130

In [47]:
final_movies_df['genre']=gen_list

In [48]:
final_movies_df['genre']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45460        [Drama, Action, Romance]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genre, Length: 45130, dtype: object

In [49]:
#dropping `genres` col
final_movies_df.drop(columns="genres" , inplace=True)

In [50]:
dummies_df = pd.get_dummies(final_movies_df['genre'].apply(pd.Series), prefix='', prefix_sep='').groupby(level=0).sum(numeric_only=True, min_count=0)



In [51]:
dummies_df

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,...,Family,Foreign,Music,Romance,Science Fiction,Thriller,Western,Comedy,Family,Thriller
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45462,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45463,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45464,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
final_movies_df = final_movies_df.join(dummies_df)

In [53]:
final_movies_df.isna().sum().sum()

0

In [54]:
len(final_movies_df[final_movies_df['original_language'] == 'en'].values)

32187

In [55]:
ratings_df['liked'] = ratings_df['rating'].apply(lambda x: 1 if x >= 3.5 else 0)


In [56]:
rtings = ratings_df['liked'].value_counts()

In [57]:
rtings.values

array([16097955,  9926334])

In [58]:

# plt.figure()
# plt.bar(x=rtings.index , height=rtings.values)
# plt.xlabel("Liked classes")
# plt.ylabel("amount of vote")
# plt.show()

model_movies_df

In [59]:
# genres_vote_count.to_csv("genres_vote_count.csv")

In [60]:
lang_s = final_movies_df["lang_list"]

In [61]:
final_movies_df.drop(columns= 'lang_list',inplace=True)

In [62]:
# lang_stacked.to_csv("language_count.csv")

In [63]:
final_movies_df

,movieId,imdb_id,original_language,original_title,overview,popularity,release_date,runtime,title,vote_average,...,Family,Foreign,Music,Romance,Science Fiction,Thriller,Western,Comedy,Family,Thriller
0,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30 00:00:00,81.0,Toy Story,7.7,...,0,0,0,0,0,0,0,0,0,0
1,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15 00:00:00,104.0,Jumanji,6.9,...,0,0,0,0,0,0,0,0,0,0
2,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22 00:00:00,101.0,Grumpier Old Men,6.5,...,0,0,0,0,0,0,0,0,0,0
3,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22 00:00:00,127.0,Waiting to Exhale,6.1,...,0,0,0,0,0,0,0,0,0,0
4,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10 00:00:00,106.0,Father of the Bride Part II,5.7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,30840,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,1991-05-13 00:00:00,104.0,Robin Hood,5.7,...,0,0,0,0,0,0,0,0,0,0
45462,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17 00:00:00,360.0,Century of Birthing,9.0,...,0,0,0,0,0,0,0,0,0,0
45463,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01 00:00:00,90.0,Betrayal,3.8,...,0,0,0,0,0,0,0,0,0,0
45464,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21 00:00:00,87.0,Satan Triumphant,0.0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
final_movies_df.to_csv("movies.csv" , index= False)
#saving cleaned movies and ready to work on 

In [65]:
ratings_df.to_csv("ratings.csv" , index=False)
#saving cleaned ratings and ready to work on 

In [66]:
ratings_df

,userId,movieId,rating,liked
0,1,110,1.0,0
1,1,147,4.5,1
2,1,858,5.0,1
3,1,1221,5.0,1
4,1,1246,5.0,1
...,...,...,...,...
26024284,270896,58559,5.0,1
26024285,270896,60069,5.0,1
26024286,270896,63082,4.5,1
26024287,270896,64957,4.5,1


In [67]:
# taking a balanced sample out of the ratings.
grouped_liked_ratings = ratings_df[ratings_df['liked']==1].sample(frac= 0.10)
grouped_not_liked_ratings = ratings_df[ratings_df['liked']==0].sample(frac= 0.16)
# ratings_sample = grouped_ratings.apply(lambda x: x.sample(grouped_ratings.size().min()).reset_index(drop=False)).sample(frac=0.15)

In [68]:
grouped_not_liked_ratings

,userId,movieId,rating,liked
17758556,184296,3113,1.0,0
12403097,128654,783,3.0,0
24574042,255443,2628,1.0,0
6029530,62067,8368,3.0,0
24694153,256682,2858,3.0,0
...,...,...,...,...
7594335,78447,94864,0.5,0
16999057,176608,3145,3.0,0
21090424,219157,4054,3.0,0
2282237,23727,1952,3.0,0


In [69]:
grouped_liked_ratings

,userId,movieId,rating,liked
22278813,231246,4128,4.0,1
24693479,256671,136538,5.0,1
20249832,210496,116797,4.0,1
17081690,177351,736,5.0,1
2723618,28263,54503,3.5,1
...,...,...,...,...
14572063,151440,2700,3.5,1
5465989,56379,42632,5.0,1
8522772,87812,1295,4.0,1
8570552,88389,1500,4.5,1


In [70]:
#sanity check 
ratings_sample =pd.concat([grouped_liked_ratings,grouped_not_liked_ratings])


In [71]:
ratings_sample =ratings_sample.sort_index()

In [72]:
ratings_sample

,userId,movieId,rating,liked
6,1,2762,4.5,1
20,1,91500,2.5,0
24,1,98809,0.5,0
31,2,64,4.0,1
39,2,648,4.0,1
...,...,...,...,...
26024204,270896,5218,3.0,0
26024257,270896,40815,4.0,1
26024280,270896,54503,4.0,1
26024282,270896,56174,3.5,1


In [75]:
#saving the sample ratings to use for modeling later 
ratings_sample.to_csv("trianing_sample.csv")

In [73]:
tfVec = TfidfVectorizer(analyzer = 'word' , stop_words='english',min_df=0,ngram_range=(1,2))

In [74]:
#sanity check on the matrix
# tfVec_matrix.shape

In [ ]:
def common_movies(user1,user2):
    movies=[]
    movies_id = [id for id in ratings_sample[ratings_sample['userId']==user1]['movieId'] if id in ratings_sample[ratings_sample['userId']==user2]['movieId']]
    # for id in movies_id:
    #     movies.append(movies_df[movies_df['movieId'] == id]['title']) 
    # return movies
    return movies_id
  

In [ ]:
#need to calculate the cosine similarity between the liked movies on users
def get_common_liked_movie(user1,user2):
    cm = common_movies(user1,user2)
    liked_by_user1 = ratings_sample[ (ratings_sample['userId']==11) & (ratings_sample['liked']==1) ]['movieId']
    liked_by_user2 = ratings_sample[(ratings_sample['userId']==22) & (ratings_sample['liked']==1)]['movieId']
    liked_movies=[]
    liked_titles=[]
    movies_id = [movie_id for movie_id in cm]
    for id in movies_id:
        if id in ((liked_by_user1) & (liked_by_user2)):
            
            liked_movies.append(id)

    for id in liked_movies:   
            liked_titles.append(movies_df[movies_df['movieId'] == id]['title'])
    return liked_titles

In [1]:
def get_liked_movies_info(movie_title):
    title = movie_title
    genre = movies_df[movies_df['movieId']==title]['genre']
    lang = movies_df[movies_df['movieId']==title]['original_language']
    runtime = movies_df[movies_df['movieId']==title]['runtime']
    

In [ ]:
#Now I'm going to vectorize the overview words.
# translator = Translator()
# def translate_non_english(text):
#     try:
#         if translator.detect(text).lang != 'en':
#             return translator.translate(text).text
#     except AttributeError:
#         pass  # Handle attribute error and return the original text
#     return text
# # Apply translation function to 'overview' column
# movies_df['overview'] = movies_df['overview'].apply(translate_non_english)

In [ ]:

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english',min_df=0,ngram_range=(1,2))

#Replace NaN with an empty string
movies_df['overview'] = movies_df['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(movies_df['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

Since we have used the TF-IDF vectorizer, calculating the dot product will directly give us the cosine similarity score. Therefore, we will use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# Function that takes in movie title as input and outputs most similar movies based on oveerview
def get_recommendations(title, cosine_sim=cosine_sim):
    
    # Get the index of the movie that matches the title
    idx = indices[title]
    print(idx)
    print()
    movie_indices =[]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    
    # Sort the movies based on the similarity scores

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices=[i[0] for i in sim_scores]
        
    # Return the top 10 most similar movies
    return movies_df['title'].iloc[movie_indices]
get_recommendations('Batman')